In [1]:
# workout_nse.py

"""Program to prepare closing (BUY) trades
Date: 23-July-2019
Ver: 1.0
Time taken: 1 min
"""

from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)
    
def workout_nse(ib):
    '''Program to dynamically prepare closing trades
    Args:
        (ib) as connection object
    Returns:
        tuple of buy contracts and orders'''

#     with get_connected('nse', 'live') as ib:
    pos = ib.positions()
    trades = ib.trades()
    pos_contracts = ib.qualifyContracts(*[Contract(conId=c) for c in [p.contract.conId for p in pos]])
    pos_tickers = ib.reqTickers(*pos_contracts)
    pos_prices = {t.contract.conId: t.marketPrice() for t in pos_tickers}
    
    if not pos:  # There is no position!
        return None # Just exit the function

    #... position dataframe
    pos_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'lastTradeDateOrContractMonth', 'strike', 'right']
    pos_df = util.df(p.contract for p in pos)[pos_cols]
    pos_df = pos_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns')
    pos_df = pos_df.assign(dte=pos_df.expiry.apply(get_dte))
    pos_df['position'] = [p.position for p in pos]
    pos_df['avgCost'] = [p.avgCost for p in pos]
    pos_df['close'] = pos_df.conId.map(pos_prices)

    # take maximum of precision and minimum of hvstPrice, avgCost
    pos_df = pos_df.assign(hvstPrice=np.maximum(np.minimum((pos_df.dte.apply(hvstPricePct)*pos_df.avgCost).apply(lambda x: get_prec(x, 0.05)), pos_df.close), prec))

    if trades:
        trades_df = util.df(t.contract for t in trades).join(util.df(t.order for t in trades)).join(util.df(t.orderStatus for t in trades), lsuffix='_')
        trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 
                       'rom', 'orderId', 'permId', 'action', 'totalQuantity', 'close', 'lmtPrice', 'status']
    else:
        trades_df = pos_df.assign(action = 'No action', status = 'No status')
        trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 
                       'rom', 'action', 'status']

    # join with other parameters in sized_nse.pkl
    trades_df = trades_df.set_index('conId').join(pd.read_pickle(fspath+'sized_nse.pkl')[['optId', 'close', 'undPrice', 'rom', 'stDev']].set_index('optId'), lsuffix='_').rename_axis('conId').reset_index()
    trades_df = trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis=1)
    trades_df = trades_df.assign(sd=abs(trades_df.strike-trades_df.undPrice)/trades_df.stDev)

    trades_df = trades_df[trades_cols]

    #... prepare BUY orders for positions without BUY orders

    # get the pos conIds that have some trade (BUY or SELL)
    pos_trade_cids = [tid for tid in trades_df.conId if tid in list(pos_df.conId)]

    # remove pos conIds that have BUY trade action
    pos_buy_cids = list(trades_df[trades_df.conId.isin(pos_trade_cids) & (trades_df.action == 'BUY') & (trades_df.status != 'Cancelled')].conId)
    pos_buy_df = pos_df[~pos_df.conId.isin(pos_buy_cids)]
    
    # make lot and qty for trade blocks and rename conId to optId
    pos_buy_df = pos_buy_df.assign(lot=pos_buy_df.position.apply(abs), qty=1, expPrice=pos_buy_df.hvstPrice)
    pos_buy_df.rename(columns={'conId': 'optId'}, inplace=True)
    
    return pos_buy_df

In [ ]:
%%time

##### script ignored by jup2py
# Program for normalizing returns from existing orders

from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)

with get_connected('nse', 'live') as ib:
    pos = ib.positions()
    trades = ib.trades()
    pos_contracts = ib.qualifyContracts(*[Contract(conId=c) for c in [p.contract.conId for p in pos]])
    pos_tickers = ib.reqTickers(*pos_contracts)
    pos_prices = {t.contract.conId: t.marketPrice() for t in pos_tickers}

#... position dataframe
pos_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'lastTradeDateOrContractMonth', 'strike', 'right']
pos_df = util.df(p.contract for p in pos)[pos_cols]
pos_df = pos_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns')
pos_df = pos_df.assign(dte=pos_df.expiry.apply(get_dte))
pos_df['position'] = [p.position for p in pos]
pos_df['avgCost'] = [p.avgCost for p in pos]
pos_df['close'] = pos_df.conId.map(pos_prices)

# take maximum of precision and minimum of hvstPrice, avgCost
pos_df = pos_df.assign(hvstPrice=np.maximum(np.minimum((pos_df.dte.apply(hvstPricePct)*pos_df.avgCost).apply(lambda x: get_prec(x, 0.05)), pos_df.close), prec))

if trades:
    trades_df = util.df(t.contract for t in trades).join(util.df(t.order for t in trades)).join(util.df(t.orderStatus for t in trades), lsuffix='_')
    trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 
                   'rom', 'orderId', 'permId', 'action', 'totalQuantity', 'close', 'lmtPrice', 'status']
else:
    trades_df = pos_df.assign(action = 'No action', status = 'No status')
    trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 
                   'rom', 'action', 'status']

# join with other parameters in sized_nse.pkl
trades_df = trades_df.set_index('conId').join(pd.read_pickle(fspath+'sized_nse.pkl')[['optId', 'close', 'undPrice', 'rom', 'stDev']].set_index('optId'), lsuffix='_').rename_axis('conId').reset_index()
trades_df = trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis=1)
trades_df = trades_df.assign(sd=abs(trades_df.strike-trades_df.undPrice)/trades_df.stDev)

trades_df = trades_df[trades_cols]

#... prepare BUY orders for positions without BUY orders

# get the pos conIds that have some trade (BUY or SELL)
pos_trade_cids = [tid for tid in trades_df.conId if tid in list(pos_df.conId)]

# remove pos conIds that have BUY trade action
pos_buy_cids = list(trades_df[trades_df.conId.isin(pos_trade_cids) & (trades_df.action == 'BUY') & (trades_df.status != 'Cancelled')].conId)
pos_buy_df = pos_df[~pos_df.conId.isin(pos_buy_cids)]

# make lot and qty for trade blocks
pos_buy_df = pos_buy_df.assign(lot=pos_buy_df.position.apply(abs), qty=1)


In [2]:
# test workout program
with get_connected('nse', 'live') as ib:
     df_buy = workout_nse(ib)

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [3]:
df_buy

,optId,symbol,localSymbol,secType,expiry,strike,right,dte,position,avgCost,close,hvstPrice,lot,qty,expPrice
0,362631153,UJJIVAN,UJJIVAN19JUL310CE,OPT,20190725,310.0,C,0,-1600.0,0.185028,0.050,0.05,1600.0,1,0.05
1,365307567,TECHM,TECHM19JUL600PE,OPT,20190725,600.0,P,0,-1200.0,0.379889,0.050,0.05,1200.0,1,0.05
15,362647741,BPCL,BPCL19JUL390CE,OPT,20190725,390.0,C,0,-1800.0,0.286556,0.050,0.05,1800.0,1,0.05
19,362667444,NMDC,NMDC19JUL107.5PE,OPT,20190725,107.5,P,0,-6000.0,0.245789,0.150,0.05,6000.0,1,0.05
22,362633898,BANKINDIA,BANKINDIA19JUL67.5PE,OPT,20190725,67.5,P,0,-6000.0,0.095956,0.050,0.05,6000.0,1,0.05
23,362625185,BERGEPAIN,BERGEPAINT19JUL350CE,OPT,20190725,350.0,C,0,-2200.0,0.288940,0.100,0.05,2200.0,1,0.05
27,362624148,INFRATEL,INFRATEL19JUL300CE,OPT,20190725,300.0,C,0,-2000.0,0.237923,0.250,0.05,2000.0,1,0.05
29,362662028,MANAPPURA,MANAPPURAM19JUL110PE,OPT,20190725,110.0,P,0,-6000.0,0.195845,0.200,0.05,6000.0,1,0.05
33,362624143,INFRATEL,INFRATEL19JUL290CE,OPT,20190725,290.0,C,0,-2000.0,0.287867,0.425,0.05,2000.0,1,0.05
37,362623879,CUMMINSIN,CUMMINSIND19JUL780CE,OPT,20190725,780.0,C,0,-700.0,0.665509,0.700,0.05,700.0,1,0.05


In [5]:
# scratchpad test
from z_helper import *
util.startLoop()

# from json
a = assign_var('nse') + assign_var('common')
for v in a:
    exec(v)
    
with get_connected('nse', 'live') as ib:
    pos = ib.positions()
    trades = ib.trades()
    pos_contracts = ib.qualifyContracts(*[Contract(conId=c) for c in [p.contract.conId for p in pos]])
    pos_tickers = ib.reqTickers(*pos_contracts)
    pos_prices = {t.contract.conId: t.marketPrice() for t in pos_tickers}
    
#... position dataframe
pos_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'lastTradeDateOrContractMonth', 'strike', 'right']
pos_df = util.df(p.contract for p in pos)[pos_cols]
pos_df = pos_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns')
pos_df = pos_df.assign(dte=pos_df.expiry.apply(get_dte))
pos_df['position'] = [p.position for p in pos]
pos_df['avgCost'] = [p.avgCost for p in pos]
pos_df['close'] = pos_df.conId.map(pos_prices)

# take maximum of precision and minimum of hvstPrice, avgCost
pos_df = pos_df.assign(hvstPrice=np.maximum(np.minimum((pos_df.dte.apply(hvstPricePct)*pos_df.avgCost).apply(lambda x: get_prec(x, 0.05)), pos_df.close), prec))

if trades:
    trades_df = util.df(t.contract for t in trades).join(util.df(t.order for t in trades)).join(util.df(t.orderStatus for t in trades), lsuffix='_')
    trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 
                   'rom', 'orderId', 'permId', 'action', 'totalQuantity', 'close', 'lmtPrice', 'status']
else:
    trades_df = pos_df.assign(action = 'No action', status = 'No status')
    trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 'undPrice', 'sd', 
                   'rom', 'action', 'status']

# join with other parameters in sized_nse.pkl
trades_df = trades_df.set_index('conId').join(pd.read_pickle(fspath+'sized_nse.pkl')[['optId', 'close', 'undPrice', 'rom', 'stDev']].set_index('optId'), lsuffix='_').rename_axis('conId').reset_index()
trades_df = trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis=1)
trades_df = trades_df.assign(sd=abs(trades_df.strike-trades_df.undPrice)/trades_df.stDev)

trades_df = trades_df[trades_cols]

#... prepare BUY orders for positions without BUY orders

# get the pos conIds that have some trade (BUY or SELL)
pos_trade_cids = [tid for tid in trades_df.conId if tid in list(pos_df.conId)]

# remove pos conIds that have BUY trade action
pos_buy_cids = list(trades_df[trades_df.conId.isin(pos_trade_cids) & (trades_df.action == 'BUY') & (trades_df.status != 'Cancelled')].conId)

pos_buy_df = pos_df[~pos_df.conId.isin(pos_buy_cids)]

In [6]:
pos_buy_df

,conId,symbol,localSymbol,secType,expiry,strike,right,dte,position,avgCost,close,hvstPrice
0,362630075,RBLBANK,RBLBANK19JUL520PE,OPT,20190725,520.0,P,3,-1200.0,2.031058,26.70,0.15
1,362650684,TORNTPOWE,TORNTPOWER19JUL300CE,OPT,20190725,300.0,C,3,-3000.0,3.938949,8.60,0.30
2,362630065,RBLBANK,RBLBANK19JUL510PE,OPT,20190725,510.0,P,3,-1200.0,2.031058,20.65,0.15
3,362625525,VOLTAS,VOLTAS19JUL670CE,OPT,20190725,670.0,C,3,-1000.0,1.178668,0.25,0.10
4,362628396,INFY,INFY19JUL810CE,OPT,20190725,810.0,C,3,-1200.0,2.830170,0.90,0.20
5,362644385,NTPC,NTPC19JUL125PE,OPT,20190725,125.0,P,3,-4800.0,1.244446,0.40,0.10
6,362629476,MUTHOOTFI,MUTHOOTFIN19JUL540PE,OPT,20190725,540.0,P,3,-1500.0,1.684780,1.70,0.10
7,362660071,BATAINDIA,BATAINDIA19JUL1200PE,OPT,20190725,1200.0,P,3,-550.0,2.760528,3.95,0.20
8,362660184,BATAINDIA,BATAINDIA19JUL1480CE,OPT,20190725,1480.0,C,3,-550.0,2.760528,0.75,0.20
9,362651611,EXIDEIND,EXIDEIND19JUL165PE,OPT,20190725,165.0,P,3,-2400.0,0.441167,0.55,0.05


In [3]:
list(trades_df)

['conId',
 'symbol',
 'localSymbol',
 'secType',
 'expiry',
 'strike',
 'right',
 'dte',
 'position',
 'avgCost',
 'close',
 'hvstPrice',
 'action',
 'status']

In [ ]:
### SOME ADDITIONAL SCRIPTS
#... see the best rom, based on latest price

# extract trades that are submitted
sub_df = trades_df[trades_df.status == 'Submitted'].drop('close', 1)

# fliter our the BUYs
sub_df = sub_df[sub_df.action == 'SELL']

# get the latest option prices
opt_list = [Option(i.symbol, i.expiry, i.strike, i.right, 'NSE') for i in sub_df[['symbol', 'expiry', 'strike', 'right']].itertuples()]

opt_contracts = []
with get_connected('nse', 'live') as ib:
    print("Qualifying option contracts ...")
    opt_contracts = ib.qualifyContracts(*opt_list)
    ticker = ib.reqTickers(*opt_contracts)

df_prices = pd.DataFrame({t.contract.conId: {'bid':t.bid, 'ask':t.ask, 'close':t.close} for t in ticker}).T

sub_df1 = sub_df.set_index('conId').join(df_prices)

sub_df1 = sub_df1.assign(lmtPrice1=(sub_df1.close+(sub_df1.ask-sub_df1.close)/2).apply(lambda x: get_prec(x, prec)))

sub_df1.assign(delta=sub_df1.lmtPrice1/sub_df1.lmtPrice).sort_values('delta', ascending=False)

In [ ]:
# prepare to place orders
buy_orders = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) 
                          for position, hvstPrice in zip(pos_buy_df.position, pos_buy_df.hvstPrice)]
buy_contracts = [{p.conId:p for p in pos_contracts}[pid] for pid in pos_buy_df.conId]